### <span style="color:#91299A">Вывод уравнений для ХКУ-приближения</span> 

<span style="color:#85c1e9">Спокойствие, только спокойствие...</span> 

![](../storage/banners/1.gif)

In [4]:
from sympy import *

t, x1, y1, z1, x0, y0, z0, vx0, vy0, vz0, w = var("t x_1 y_1 z_1 x_0 y_0 z_0 v^x_0 v^y_0 v^z_0 w_0")
c1 = 2 * z0 + vx0/w
c2 = vz0/w
c3 = -3 * z0 - 2 * vx0/w
c4 = x0 - 2 * vz0/w
c5 = vy0/w
c6 = y0
eq = [Eq(x1, c4 - 3*c1*w*t + 2*c2*cos(w*t) - 2*c3*sin(w*t)),
      Eq(y1, c5*sin(w*t) + c6*cos(w*t)),
      Eq(z1, 2*c1 + c2*sin(w*t) + c3*cos(w*t)),]
anw = solve(eq, [vx0, vy0, vz0])
anw

{v^x_0: -6*t*w_0**2*z_0*sin(t*w_0)/(3*t*w_0*sin(t*w_0) - 4*sin(t*w_0)**2 - 4*cos(t*w_0)**2 + 8*cos(t*w_0) - 4) + w_0*x_0*sin(t*w_0)/(3*t*w_0*sin(t*w_0) - 4*sin(t*w_0)**2 - 4*cos(t*w_0)**2 + 8*cos(t*w_0) - 4) - w_0*x_1*sin(t*w_0)/(3*t*w_0*sin(t*w_0) - 4*sin(t*w_0)**2 - 4*cos(t*w_0)**2 + 8*cos(t*w_0) - 4) + 6*w_0*z_0*sin(t*w_0)**2/(3*t*w_0*sin(t*w_0) - 4*sin(t*w_0)**2 - 4*cos(t*w_0)**2 + 8*cos(t*w_0) - 4) + 6*w_0*z_0*cos(t*w_0)**2/(3*t*w_0*sin(t*w_0) - 4*sin(t*w_0)**2 - 4*cos(t*w_0)**2 + 8*cos(t*w_0) - 4) - 14*w_0*z_0*cos(t*w_0)/(3*t*w_0*sin(t*w_0) - 4*sin(t*w_0)**2 - 4*cos(t*w_0)**2 + 8*cos(t*w_0) - 4) + 8*w_0*z_0/(3*t*w_0*sin(t*w_0) - 4*sin(t*w_0)**2 - 4*cos(t*w_0)**2 + 8*cos(t*w_0) - 4) + 2*w_0*z_1*cos(t*w_0)/(3*t*w_0*sin(t*w_0) - 4*sin(t*w_0)**2 - 4*cos(t*w_0)**2 + 8*cos(t*w_0) - 4) - 2*w_0*z_1/(3*t*w_0*sin(t*w_0) - 4*sin(t*w_0)**2 - 4*cos(t*w_0)**2 + 8*cos(t*w_0) - 4),
 v^y_0: -w_0*y_0*cos(t*w_0)/sin(t*w_0) + w_0*y_1/sin(t*w_0),
 v^z_0: -3*t*w_0**2*z_0*cos(t*w_0)/(3*t*w_0*sin(t*w_0)

In [5]:
anw[vx0].simplify()

w_0*(6*t*w_0*z_0*sin(t*w_0) - x_0*sin(t*w_0) + x_1*sin(t*w_0) + 14*z_0*cos(t*w_0) - 14*z_0 - 2*z_1*cos(t*w_0) + 2*z_1)/(-3*t*w_0*sin(t*w_0) - 8*cos(t*w_0) + 8)